In [1]:
from btree import BTree
class KVStore(BTree):
    def __init__(self,split_threshold=2):
        super().__init__(split_threshold)
        

In [2]:
#override the add method of BTree not to add multiple entries with same key
class KVStore(BTree):
    def __init__(self,split_threshold=2):
        super().__init__(split_threshold)
    def add(self, key, value):
        node = self._find_node(self.root, key)
        if node is None:
            super().add(key, value)
        else:
            for i, node_key in enumerate(node.keys):
                if node_key == key:
                    node.values[i] = value

In [3]:
#test
kv = KVStore()
assert kv.split_threshold == 2, "The split is not equal to 2."

for i in range(10):
    kv.add(i, i + 1)
for i in range(10):
    assert kv.get_value(i) == i + 1, "The value of i is not i + 1"

In [4]:
kv = KVStore()
kv.add(2, 10)
print(kv.get_value(2))

10


make a function to use brackets eg: kv[2] = 10 instead

In [5]:
class KVStore(BTree):
    def __init__(self,split_threshold=2):
        super().__init__(split_threshold)
    def add(self, key, value):
        node = self._find_node(self.root, key)
        if node is None:
            super().add(key, value)
        else:
            for i, node_key in enumerate(node.keys):
                if node_key == key:
                    node.values[i] = value
                    
    #enhance get value -> get item method
    def __getitem__(self,key):
        return self.get_value(key)
    #enhance add value -> set item method
    def __setitem__(self,key,value):
        self.add(key,value)

In [6]:
#test
kv = KVStore()
kv[2] = 10
print(kv[2])

kv[2] = 20
print(kv[2])

10
20


In [7]:
#enhance contain method
class KVStore(BTree):
    def __init__(self,split_threshold=2):
        super().__init__(split_threshold)
    def add(self, key, value):
        node = self._find_node(self.root, key)
        if node is None:
            super().add(key, value)
        else:
            for i, node_key in enumerate(node.keys):
                if node_key == key:
                    node.values[i] = value
                    
    def __setitem__(self, key, value):
        self.add(key, value)
        
    def __getitem__(self, key):
        return self.get_value(key)
    #enhanced contain method
    def __contains__(self, key):
        return self.contains(key)

In [8]:
#test if can use the "in" operator on an instance of KVStore
kv = KVStore()
for e in 'hello':
    kv[e] = e

for e in 'hello':
    assert e in kv, "Character is not in the kvs"

In [9]:
#add range query method
class KVStore(BTree):
    def __init__(self,split_threshold=2):
        super().__init__(split_threshold)
    def add(self, key, value):
        node = self._find_node(self.root, key)
        if node is None:
            super().add(key, value)
        else:
            for i, node_key in enumerate(node.keys):
                if node_key == key:
                    node.values[i] = value
                    
    def __setitem__(self, key, value):
        self.add(key, value)
        
    def __getitem__(self, key):
        return self.get_value(key)

    def __contains__(self, key):
        return self.contains(key)
    #add range query method
    def _range_query(self, range_start, range_end, current_node, min_key, max_key):
        if range_start > max_key or range_end < min_key:
            return []
        results = []
        for i, key in enumerate(current_node.keys):
            if range_start <= key and key <= range_end:
                results.append(current_node.values[i])
        if not current_node.is_leaf():
            for i, child in enumerate(current_node.children):
                new_min_key = current_node.keys[i - 1] if i > 0 else min_key
                new_max_key = current_node.keys[i] if i < len(current_node) else max_key
                results += self._range_query(range_start, range_end, child, new_min_key, new_max_key)
        return results 

    def range_query(self, range_start, range_end):
        return self._range_query(range_start, range_end, self.root, float('-inf'), float('inf'))

In [10]:
class DictKVStore(dict):
    
    def range_query(self, range_start, range_end):
        result = []
        for key in self.keys():
            if range_start <= key and key <= range_end:
                result.append(self[key])
        return result

In [11]:
#test
dict_kv = DictKVStore()
our_kv = KVStore()
for i in range(100):
    dict_kv[i] = i
    our_kv[i] = i

for range_start, range_end in [(1, 3), (5, 5)]:
    dict_res = sorted(dict_kv.range_query(range_start, range_end))
    our_res = sorted(our_kv.range_query(range_start, range_end))
    assert dict_res == our_res, "Both data structures return the same range query result."

test implementation of KVStore with DictKVStore as baseline

In [12]:
import random
random.seed(0)

dict_kv = DictKVStore()

kv = KVStore()

print("Testing Insertion")
for _ in range(10):
    key = random.randint(0, 100)
    value = random.randint(0, 1000000)
    dict_kv[key] = value
    kv[key] = value
    
print("Testing Length")
assert len(dict_kv) == len(kv), "Wrong length. Length should be {} but is {}.".format(len(dict_kv), len(kv))
    

print("Testing Values")
for key in dict_kv:
    assert dict_kv[key] == kv[key], "Wrong value for key {}. Expected value {} but found value {}.".format(key, dict_kv[key], kv[key])

    
print("Testing in Operator")
for i in range(10):
    key = random.randint(0, 1000)
    assert (key in dict_kv) == (key in kv), "Contains method did not return the correct value for key {}.".format(key)


print("Testing Range Queries")
for _ in range(10):
    range_start = random.randint(0, 100)
    range_end = random.randint(range_start, 100)
    dict_results = dict_kv.range_query(range_start, range_end)
    kv_results = kv.range_query(range_start, range_end)
    assert len(dict_results) == len(kv_results), "Wrong number of reuslt in range query [{}, {}]. Should be {} but was {}.".format(range_start, range_end, len(dict_result), len(kv_result))
    dict_results.sort()
    kv_results.sort()
    assert dict_results == kv_results, "Wrong number of reuslt in range query [{}, {}]. Should be {} but was {}.".format(range_start, range_end, len(dict_result), len(kv_result))


Testing Insertion
Testing Length
Testing Values
Testing in Operator
Testing Range Queries
